# Outline

The Julia DataFrames package is a handy package for working with and manipulating tabular data in Julia. It's well suited for working with data where the columns are of different types, i.e. heterogeneous data, and when the dataset can fit in memory. It can be used to perform a variety of data manipulation operations such as subsetting rows, selecting columns, performing aggregations by group, joining, etc. We will explore doing all of these things and more.

What we'll be covering today:

#### I. Getting started
#### II. Working with dataframes
#### III. Joining and concatenating
#### IV. Handling missing values
#### V. Split-apply-combine
#### VI. Using Query.jl

# I. Getting started

In [ ]:
#To install
#using Pkg
#Pkg.add("DataFrames")

In [ ]:
#To load the DataFrames package once installed
using DataFrames

### Dataframes fundamentals:

The basic data structure you will be working with is the **DataFrame**. This type is defined in the DataFrames package. In this section we'll see a few different ways of manually creating dataframes using the DataFrame constructor. You'll rarely use this constructor directly to create your dataframes, except for maybe testing out ideas, but it's good to have an understanding of how to do this.


Let's start by creating a DataFrame explicitly using keyword arguments. We'll create a small dataframe **df** with five columns named <i>A</i> , <i>B</i>, <i>C</i>, <i>D</i>, and <i>E</i>.

In [ ]:
using Random

In [ ]:
df = DataFrame(B = [0, 1, 1, 0], C = [0, 0, 1, 1], A = [0, 1, 0, 1], D = [randstring(9) for j in 1:4], E = 1:4)

In [ ]:
typeof(df)

Note that column names in Julia are actually **Symbols** and not **Strings**. In Julia, symbols are prefixed with ":" which is how you can tell that an object is a symbol. Also notice that Julia has typed each colulmn.

For us it's not too important to know what a symbol is exactly in Julia. You just need to be aware that when referring to columns in your dataframes you will need to refer to the column names as symbols (using the symbol notation) and not strings.

You can initialize an empty dataframe using the `DataFrame()` constructor and then build it up using a dictionary or by arbitrarily adding columns. In this case you pass the dictionary as an argument to the `DataFrame()` constructor.

In [ ]:
df = DataFrame(); #initialize an empty dataframe

In [ ]:
d = Dict("A" => [0, 1, 0, 1], "B" => [0, 1, 1, 0], "C" => [0, 0, 1, 1], "D" => [randstring(9) for j in 1:4], 
    "E" => 1:4)

In [ ]:
df = DataFrame(d)

The following syntax also works where you pass the dataframe constructor a comma-separated list of **Pairs** where the first element of each pair is a **Symbol** that refers to the column and the second element are the values. Note with this method the order in which the columns were passed was maintained in the resulting dataframe.

In [ ]:
df = DataFrame(:B => [0, 1, 1, 0], :A => [0, 1, 0, 1], :C => [0, 0, 1, 1], :D => [randstring(9) for j in 1:4], 
    :E => 1:4)

You can build up an empty dataframe by explicitly adding columns using dot notation to refer to columns.

In [ ]:
df = DataFrame();
df.B = [0, 1, 1, 0];
df.C = [0, 0, 1, 1];
df.A = [0, 1, 0, 1];
df.D = [randstring(9) for j in 1:4];
df.E = 1:4;

In [ ]:
df

You can also create a dataframe by passing in the column values and symbols as separate arguments to the `DataFrame()` constructor. The first argument is an array of vectors where each vector is a column of data; the second argument is the array of symbols designating the column names.

In [ ]:
df = DataFrame([[0,1,1,0], [0,0,1,1], [0,1,0,1], [randstring(9) for j in 1:4], 1:4], [:B, :C, :A, :D, :E])

If you want to convert your dataframe to an array wrap the dataframe in a call to `Matrix`.

In [ ]:
m = Matrix(df)

And then to convert it back to a dataframe you can wrap the array in a call to `DataFrame()`. Note the arbitrary column names <i>x1</i>, <i>x2</i>, etc.

In [ ]:
df = DataFrame(m)

Finally, if you need to, you can initialize a non-empty datframe with garbage values. You will need to specify the desired columns types and optionally specify the column names and number of rows.

In [ ]:
df_garbage = DataFrame([Int64, Int64, Int64, String, Int64], [:B, :C, :A, :D, :E], 4)

Regardless of how you create it, the **DataFrame** type represents a data table as a series of vectors, each corresponding to a column or variable.

### Selecting columns:

In [ ]:
# create a dataframe

df = DataFrame(A = [0, 1, 0, 1], B = [0, 1, 1, 0], C = [0, 0, 1, 1], D = [randstring(9) for j in 1:4], E = 1:4);

You can select individual columns in a few different ways:
- `df.col`
- `df."col"`
- `df[!,:col]`
- `df[!, col_idx]`


In [ ]:
df

Below we access columns <i>A</i>, <i>B</i>, and <i>C</i> using the dot notation. Note this does __not__ return a copy of the column data; so if you modify __df.A__ then you will modify the dataframe as well.

In [ ]:
df.A

In [ ]:
typeof(df.A)

In [ ]:
typeof(df.D)

You can also use string notation again this does __not__ return a copy:

In [ ]:
df."A"

You can refer to columns using bracket notation but note that for the column you have to use the symbol notation, i.e., :B and not "B". The ! used above means to grab all rows.

Please note that `df[!, :col]` does **not** make a copy of the column therefore modifying elements in it will change elements in the dataframe itself. If you want to work with a copy use `df[:, :col]`.

In [ ]:
df[!, :B][2] = 2# df[!, "B"] will not work

In [ ]:
a = df[!, :B]

In [ ]:
df

You can also use the column index to refer to a specific column. Here we get the third column. Note that indexing in Julia starts at 1.

In [ ]:
df[!, 3] #third way using a column index

You can retrieve multiple columns by listing them out by symbol or index. In this case the returned object will be a dataframe.

In [ ]:
df[!, [:A, :D]] # get columns A and D

In [ ]:
df[!, 2:5]  #get columns two through five

An alternative method to selecting columns in a dataframe is to use the `select` function.

In [ ]:
select(df, :A) #select column A

In [ ]:
select(df, [:A, :D]) # select columns A and D

In [ ]:
select(df, 2:5) #select columns 2 through 5

In [ ]:
select(df, Not(:A)) #select all columns except column A

The `select` function returns a new dataframe where the columns are copies of the columns from the original dataframe.

To get the names and types of the columns you can use the `names` and `eltype` functions.

In [ ]:
names(df)

In [ ]:
eltype.(eachcol(df))

In [ ]:
?eachcol

You can append a row using `push!` and providing the row values in a tuple.

In [ ]:
push!(df, (1, 1, 1, randstring(7), 5))

### Reading and writing data:

Most likely you will not be manually creating dataframes as above but rather loading data from external files.

You can read and write data to a variety of file formats.

If you want to save your dataframe to a CSV file you can use the `CSV.write` function in the **CSV.jl** package:

In [ ]:
using CSV

The first argument is the desired name of the CSV file and the second is the name of the dataframe:

In [ ]:
CSV.write("mydf.csv", df);

If you want to load the CSV file use `CSV.read`:

In [ ]:
df = CSV.read("mydf.csv")

In [ ]:
typeof(df)

Note that the column type is different for a dataframe created from reading in a csv file versus a dataframe created manually.

In [ ]:
typeof(df.A)

You can specify saving the dataframe using a different delimiter:

In [ ]:
CSV.write("mydf.tsv", df, delim='\t');

In [ ]:
df = CSV.read("mydf.tsv", delim='\t');

In [ ]:
df

The CSV.jl package has a couple of useful features. Let us look at a few features related to reading in csv data (there are many more than we'll cover here).

You can indicate where the header row starts in the file. By default, data will be read in starting on the next row. Let's look at our example file:

In [ ]:
;cat readinexample.csv

Here the header row is on the third line so we can specify that in our CSV.read() command using its `header` keyword argument.

In [ ]:
CSV.read("readinexample.csv", header=3)

 Note if your file has no header you can simply set `header=false`.

You can also read in data starting at a specified row in the file. There are two ways to do this: one way is using the `datarow` keyword argument and anoterway is via the `skipto` keyword argument. The former indicates the row number at which to start reading in data; the latter indicates the number of rows to skip before reading in data.

Here we indicate the header is on row 3 and the data we want to read starts on row 6.

In [ ]:
CSV.read("readinexample.csv", header=3, datarow=6)

In [ ]:
CSV.read("readinexample.csv", header=3, skipto=6)

If certain values should be treated as `missing` you can indicate that with the `missingstrings` keyword argument. In our file, let's assume that the values 99 and NA should be treated as missing when the data is read in:

In [ ]:
CSV.read("readinexample.csv", header=3, missingstrings=["99", "NA"])

The last thing we'll cover is selecting specific columns, or dropping columns, when reading in data. Suppose we only wanted to read in the columns _A_, _B_, and _D_? You can specify this using the column name or column index in the `select` keyword argument.

In [ ]:
CSV.read("readinexample.csv", header=3, select=["A", "B", "D"])

And if you wanted to drop columns _C_ and _D_ use `drop`:

In [ ]:
CSV.read("readinexample.csv", header=3, drop=[C", "D"])

You can also use the column index number. Here columns _C_ and _D_ are the third and fifth columns respectively in our csv file:

In [ ]:
CSV.read("readinexample.csv", header=3, drop=[3,5])

There are other Julia packages for reading other file formats (these are just a select few):
* ReadStat.jl: Stata, SAS, and SPSS data files.
* Parquet.jl: Parquet files.
* JSON.jl, JSON2.jl, JSON3.jl: JSON files.


In this lesson we covered:
* What the Julia DataFrames package can be used for.
* What the DataFrame type is.
* The basics of Julia datframes.
* Simple I/O using dataframes and the CSV.jl package.